In [17]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from enum import Enum
from copy import copy
from scipy.stats import norm
from scipy.cluster import hierarchy as sch
import pandas as pd
import numpy as np

import yfinance as yf

import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from _utils.core_functions import *
from _utils.portfoliohandcrafiting import *
from _utils.strategies.trend_simple_filter import *


DATA_DIR = "../_databases"

DEFAULT_DATE_FORMAT = "%Y-%m-%d"


In [18]:
#%% GET DATA
# # =================

def get_data_dict(instrument_list: list):

    all_data = dict(
        [
            (instrument_code, pd.read_csv((DATA_DIR + '/' + f'{instrument_code}' + '.csv'),index_col='index'))
            for instrument_code in instrument_list
        ]
    )

    adjusted_prices = dict(
        [
            (instrument_code, data_for_instrument.adjusted)
            for instrument_code, data_for_instrument in all_data.items()
        ]   
    )

    current_prices = dict(
        [
            (instrument_code, data_for_instrument.underlying)
            for instrument_code, data_for_instrument in all_data.items()
        ]
    )

    return adjusted_prices, current_prices

In [19]:
us_adjusted_prices, current_prices = get_data_dict(['sp500','us10'])
multipliers = dict(sp500=5, us10=1000)
risk_target_tau = 0.2

fx_series_dict = create_fx_series_given_adjusted_prices_dict(us_adjusted_prices,dict(sp500=1, us10=1))

capital = 1000000
idm = 1.5
instrument_weights = dict(sp500=0.5, us10=0.5)

std_dev_dict = calculate_variable_standard_deviation_for_risk_targeting_from_dict(
adjusted_prices=us_adjusted_prices,
current_prices=current_prices,
annualise_stdev=True,  ## can also be False if want to use daily price diff
use_perc_returns=True,  ## can also be False if want to use daily price diff
)

In [20]:

# 50% - 50%  US portfolio with EWMAC long only filter
us_average_position_contracts_dict = (
        calculate_position_series_given_variable_risk_for_dict(
            capital=capital,
            risk_target_tau=risk_target_tau,
            idm=idm,
            weights=instrument_weights,
            std_dev_dict=std_dev_dict,
            fx_series_dict=fx_series_dict,
            multipliers=multipliers,
        )
    )

us_position_contracts_trend_forecast_dict = calculate_position_dict_with_trend_forecast_applied(
        adjusted_prices_dict=us_adjusted_prices,
        average_position_contracts_dict=us_average_position_contracts_dict,
        std_dev_dict=std_dev_dict,
        fast_span=64
    )


us_position_contracts_filter_dict = calculate_position_dict_with_symmetric_trend_filter_applied(
        adjusted_prices_dict=us_adjusted_prices,
        average_position_contracts_dict=us_average_position_contracts_dict,
    )

us_position_contracts_longonly_dict = calculate_position_dict_with_trend_filter_applied(
        adjusted_prices_dict=us_adjusted_prices,
        average_position_contracts_dict=us_average_position_contracts_dict,
    )



us_perc_return_trend_forecast_dict = calculate_perc_returns_for_dict(
position_contracts_dict=us_position_contracts_trend_forecast_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=us_adjusted_prices,
)

us_perc_return_filter_dict = calculate_perc_returns_for_dict(
position_contracts_dict=us_position_contracts_filter_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=us_adjusted_prices,
)

us_perc_return_longonly_dict = calculate_perc_returns_for_dict(
position_contracts_dict=us_position_contracts_longonly_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=us_adjusted_prices,
)

us_perc_return_dict = calculate_perc_returns_for_dict(
position_contracts_dict=us_average_position_contracts_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=us_adjusted_prices,
)

us_port_trend_forecast_return = aggregate_returns(us_perc_return_trend_forecast_dict)
us_port_trend_forecast_return.sort_index(inplace=True)


us_port_filter_return = aggregate_returns(us_perc_return_filter_dict)
us_port_filter_return.sort_index(inplace=True)

us_port_longonly_return = aggregate_returns(us_perc_return_longonly_dict)
us_port_longonly_return.sort_index(inplace=True)

us_port_return = aggregate_returns(us_perc_return_dict)
us_port_return.sort_index(inplace=True)



In [21]:
# --- Plot retorno acumulado
fig = go.Figure()


fig.add_trace(
    go.Scatter(
        x=us_port_trend_forecast_return.index,
        y=us_port_trend_forecast_return.cumsum()*100,
        name="EWMAC with Forecast - Long",
        line=dict(color="black"),
    )
)

fig.add_trace(
    go.Scatter(
        x=us_port_filter_return.index,
        y=us_port_filter_return.cumsum()*100,
        name="EWMAC Portfolio - Long Short",
        line=dict(color="blue"),
    )
)

fig.add_trace(
    go.Scatter(
        x=us_port_longonly_return.index,
        y=us_port_longonly_return.cumsum()*100,
        name="EWMAC Portfolio - Long Only",
        line=dict(color="red"),
    )
)

fig.add_trace(
    go.Scatter(
        x=us_port_return.index,
        y=us_port_return.cumsum()*100,
        name="Portfolio without filter",
        line=dict(color="green"),
    )
)

fig.update_layout(
    title="50% SP500 Fut + 50% US10 Fut Portfolio",
    xaxis_title="Data",
    yaxis_title="Acumulated return (%)",
    template="plotly_white",
    showlegend=True,
    legend=dict(
        yanchor="bottom",
        y=0.5,
        xanchor="left",
        x=0),
    height=600,
    width=1000,
)

fig.show()

# --- Exibir estatísticas
print("Estatísticas de Desempenho")
print("Estatísticas Portfolio EWMAC com filtro de Trend Frequência Anual")
print(calculate_stats(us_port_trend_forecast_return,freq=YEAR))
print("================================")
print("Estatísticas Portfolio EWMAC Long and Short na Frequência Anual")
print(calculate_stats(us_port_filter_return, freq=YEAR))
print("================================")
print("Estatísticas Portfolio EWMAC Long Only Frequência Anual")
print(calculate_stats(us_port_longonly_return,freq=YEAR))
print("================================")
print("Estatísticas Portfolio sem EWMAC na Frequência Anual")
print(calculate_stats(us_port_return, freq=YEAR))


Estatísticas de Desempenho
Estatísticas Portfolio EWMAC com filtro de Trend Frequência Anual
{'ann_mean': 40.46439279198572, 'ann_std': 3.774762747779584, 'sharpe': 10.719718163952942, 'skew': 0.7140938493390666, 'avg_drawdown': 0.037813362857657136, 'max_drawdown': 0.27652263014065026, 'quant_lower': 0.573898445120583, 'quant_upper': 1.4672142729959652}
Estatísticas Portfolio EWMAC Long and Short na Frequência Anual
{'ann_mean': 42.87961369540085, 'ann_std': 3.2630136854391756, 'sharpe': 13.14110752484619, 'skew': -0.1256034722095622, 'avg_drawdown': 0.026023463539547233, 'max_drawdown': 0.3187455897058773, 'quant_lower': 0.7103289483388047, 'quant_upper': 0.8839417051542012}
Estatísticas Portfolio EWMAC Long Only Frequência Anual
{'ann_mean': 46.33961263740601, 'ann_std': 3.181713205956037, 'sharpe': 14.564358770822007, 'skew': 0.37360972392803393, 'avg_drawdown': 0.013123591406956242, 'max_drawdown': 0.25455224635742846, 'quant_lower': 0.6870019320315264, 'quant_upper': 1.3176968911

In [22]:
br_adjusted_prices, current_prices = get_data_dict(['winfut','wdofut'])
multipliers = dict(winfut=5, wdofut=1000)
risk_target_tau = 0.2

fx_series_dict = create_fx_series_given_adjusted_prices_dict(br_adjusted_prices,dict(winfut=1, wdofut=1))

capital = 1000000
idm = 1.5
instrument_weights = dict(winfut=0.5, wdofut=0.5)

std_dev_dict = calculate_variable_standard_deviation_for_risk_targeting_from_dict(
adjusted_prices=br_adjusted_prices,
current_prices=current_prices,
annualise_stdev=True,  ## can also be False if want to bre daily price diff
use_perc_returns=True,  ## can also be False if want to bre daily price diff
)


# 50% - 50%  br portfolio with EWMAC long only filter
br_average_position_contracts_dict = (
        calculate_position_series_given_variable_risk_for_dict(
            capital=capital,
            risk_target_tau=risk_target_tau,
            idm=idm,
            weights=instrument_weights,
            std_dev_dict=std_dev_dict,
            fx_series_dict=fx_series_dict,
            multipliers=multipliers,
        )
    )

br_position_contracts_trend_forecast_dict = calculate_position_dict_with_trend_forecast_applied(
        adjusted_prices_dict=br_adjusted_prices,
        average_position_contracts_dict=br_average_position_contracts_dict,
        std_dev_dict=std_dev_dict,
        fast_span=64
    )


br_position_contracts_filter_dict = calculate_position_dict_with_symmetric_trend_filter_applied(
        adjusted_prices_dict=br_adjusted_prices,
        average_position_contracts_dict=br_average_position_contracts_dict,
    )

br_position_contracts_longonly_dict = calculate_position_dict_with_trend_filter_applied(
        adjusted_prices_dict=br_adjusted_prices,
        average_position_contracts_dict=br_average_position_contracts_dict,
    )

br_perc_return_trend_forecast_dict = calculate_perc_returns_for_dict(
position_contracts_dict=br_position_contracts_trend_forecast_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=br_adjusted_prices,
)

br_perc_return_filter_dict = calculate_perc_returns_for_dict(
position_contracts_dict=br_position_contracts_filter_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=br_adjusted_prices,
)

br_perc_return_longonly_dict = calculate_perc_returns_for_dict(
position_contracts_dict=br_position_contracts_longonly_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=br_adjusted_prices,
)

br_perc_return_dict = calculate_perc_returns_for_dict(
position_contracts_dict=br_average_position_contracts_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=br_adjusted_prices,
)


br_port_trend_forecast_return = aggregate_returns(br_perc_return_trend_forecast_dict)
br_port_trend_forecast_return.sort_index(inplace=True)

br_port_filter_return = aggregate_returns(br_perc_return_filter_dict)
br_port_filter_return.sort_index(inplace=True)

br_port_longonly_return = aggregate_returns(br_perc_return_longonly_dict)
br_port_longonly_return.sort_index(inplace=True)

br_port_return = aggregate_returns(br_perc_return_dict)
br_port_return.sort_index(inplace=True)






In [23]:
# --- Plot retorno acumulado
fig = go.Figure()


fig.add_trace(
    go.Scatter(
        x=br_port_trend_forecast_return.index,
        y=br_port_trend_forecast_return.cumsum()*100,
        name="EWMAC with Forecast - Long",
        line=dict(color="black"),
    )
)


fig.add_trace(
    go.Scatter(
        x=br_port_filter_return.index,
        y=br_port_filter_return.cumsum()*100,
        name="EWMAC Portfolio - Long Short",
        line=dict(color="blue"),
    )
)

fig.add_trace(
    go.Scatter(
        x=br_port_longonly_return.index,
        y=br_port_longonly_return.cumsum()*100,
        name="EWMAC Portfolio - Long Only",
        line=dict(color="red"),
    )
)

fig.add_trace(
    go.Scatter(
        x=br_port_return.index,
        y=br_port_return.cumsum()*100,
        name="Portfolio without filter",
        line=dict(color="green"),
    )
)

fig.update_layout(
    title="50% WINFUT + 50% WDOFUT Portfolio",
    xaxis_title="Data",
    yaxis_title="Acumulated return (%)",
    template="plotly_white",
    showlegend=True,
    legend=dict(
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1),
    height=600,
    width=1000,
)

fig.show()

# --- Exibir estatísticas
print("Estatísticas de Desempenho")
print("Estatísticas Portfolio EWMAC com filtro de Trend Long Frequência Anual")
print(calculate_stats(br_port_trend_forecast_return,freq=YEAR))
print("================================")
print("Estatísticas Portfolio EWMAC Long and Short na Frequência Anual")
print(calculate_stats(br_port_filter_return, freq=YEAR))
print("================================")
print("Estatísticas Portfolio EWMAC Long Only Frequência Anual")
print(calculate_stats(br_port_longonly_return,freq=YEAR))
print("================================")
print("Estatísticas Portfolio sem EWMAC na Frequência Anual")
print(calculate_stats(br_port_return, freq=YEAR))

Estatísticas de Desempenho
Estatísticas Portfolio EWMAC com filtro de Trend Long Frequência Anual
{'ann_mean': -3.6355534866509043, 'ann_std': 2.2844481326025083, 'sharpe': -1.5914362137472469, 'skew': 1.5931378764022142, 'avg_drawdown': 0.38939887852705807, 'max_drawdown': 0.8059694069838237, 'quant_lower': 0.9749126259838604, 'quant_upper': 6.813536241226416}
Estatísticas Portfolio EWMAC Long and Short na Frequência Anual
{'ann_mean': 2.9932101720489723, 'ann_std': 2.4816859512932985, 'sharpe': 1.2061196423701797, 'skew': 1.2464601791256973, 'avg_drawdown': 0.18843951043710958, 'max_drawdown': 0.47074625938214476, 'quant_lower': 0.8810498123128881, 'quant_upper': 1.8101301056400065}
Estatísticas Portfolio EWMAC Long Only Frequência Anual
{'ann_mean': 8.53681626030101, 'ann_std': 1.6690091464701267, 'sharpe': 5.114900825052973, 'skew': 0.7800004761083987, 'avg_drawdown': 0.07009679449534045, 'max_drawdown': 0.22603903452819402, 'quant_lower': 0.3447279103188287, 'quant_upper': 0.94494